* SCB Issue [#41](https://github.com/salgo60/SCB-Wikidata/issues/41)
* this Notebook [SCB41_matchWD.ipynb](https://github.com/salgo60/SCB-Wikidata/tree/main/notebook/SCB41_matchWD.ipynb)

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-11-25 16:17:43


In [3]:
# Notebook-style script (works well as a Jupyter / jupytext Python notebook)
# Purpose: Koppla ihop Myndighetsreg_alla.csv med Wikidata och skriva ut Q-id (wikidata_qid),
#          wikidata_label, match_method och match_score.
#
# Kör i en virtuell miljö där du har pandas och requests installerade:
# pip install pandas requests tqdm

# %% Imports & inställningar
import pandas as pd
import requests
import time
import math
from difflib import SequenceMatcher
from urllib.parse import quote
from tqdm import tqdm

WIKIDATA_SPARQL = "https://query.wikidata.org/sparql"
WIKIDATA_API = "https://www.wikidata.org/w/api.php"

RAW_CSV_URL = "https://raw.githubusercontent.com/salgo60/SCB-Wikidata/refs/heads/main/notebook/Myndighetsreg_concat/Myndighetsreg_alla.csv"

# Anpassa sleep för att undvika rate-limits
SLEEP_BETWEEN_REQUESTS = 0.5

# %% Hjälpfunktioner för matchning

def similarity(a: str, b: str) -> float:
    if not a or not b:
        return 0.0
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def sparql_exact_label_search(name: str, lang: str = "sv"):
    """
    Söker efter Wikidata-items med exakt rdfs:label == name@lang via SPARQL.
    Returnerar lista av dict: [{'qid': 'Qxxx', 'label': '...', 'uri': '...'}, ...]
    """
    # Escape quotes
    name_escaped = name.replace('"', '\\"')
    q = f'''
    SELECT ?item ?itemLabel WHERE {{
      ?item rdfs:label "{name_escaped}"@{lang} .
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "{lang},en". }}
    }} LIMIT 20
    '''
    headers = {"Accept": "application/sparql-results+json", "User-Agent": "salgo60-matching-script/1.0 (https://github.com/salgo60/SCB-Wikidata)"}
    resp = requests.get(WIKIDATA_SPARQL, params={"query": q}, headers=headers, timeout=60)
    time.sleep(SLEEP_BETWEEN_REQUESTS)
    resp.raise_for_status()
    data = resp.json()
    results = []
    for row in data.get("results", {}).get("bindings", []):
        uri = row["item"]["value"]
        qid = uri.rsplit("/", 1)[-1]
        label = row.get("itemLabel", {}).get("value", "")
        results.append({"qid": qid, "label": label, "uri": uri})
    return results

def wikidata_api_search(name: str, lang: str = "sv", limit: int = 10):
    """
    Använder wbsearchentities för att få candidates baserat på name.
    Returnerar lista av dict: [{'qid':'Qxxx','label':'...','description':'...', 'match':score_estimate}, ...]
    """
    params = {
        "action": "wbsearchentities",
        "format": "json",
        "language": lang,
        "search": name,
        "limit": limit
    }
    headers = {"User-Agent": "salgo60-matching-script/1.0 (https://github.com/salgo60/SCB-Wikidata)"}
    resp = requests.get(WIKIDATA_API, params=params, headers=headers, timeout=60)
    time.sleep(SLEEP_BETWEEN_REQUESTS)
    resp.raise_for_status()
    data = resp.json()
    results = []
    for item in data.get("search", []):
        results.append({
            "qid": item.get("id"),
            "label": item.get("label"),
            "description": item.get("description", "")
        })
    return results

# %% Hämta CSV och förbered DataFrame
df = pd.read_csv(RAW_CSV_URL, dtype=str, low_memory=False)
# Normalisera kolumn med namn - anta kolumnen heter "Namn" (som i ditt CSV-exempel).
# Om annan kolumn, ändra nedan.
NAME_COL = "Namn"
if NAME_COL not in df.columns:
    raise ValueError(f"Förväntar kolumnen '{NAME_COL}' i CSV, men hittade inte. Kolumner: {list(df.columns)}")

df[NAME_COL] = df[NAME_COL].astype(str).str.strip()
# Förbered nya kolumner
df["wikidata_qid"] = ""
df["wikidata_label"] = ""
df["match_method"] = ""
df["match_score"] = ""

# %% Cache för att undvika upprepade anrop för samma söksträng
cache_exact = {}
cache_api = {}

# %% Kör matchningen (exempel: begränsat antal rader vid test - ta bort head() för hela filen)
# För produktion: iterate över df.itertuples()
rows = list(df.index)  # hela datasetet
# Om du vill testa först, byt till rows = df.index[:200]

for i in tqdm(rows, desc="Matching rows"):
    name = df.at[i, NAME_COL]
    if not name or name.strip() == "" or name.lower() in ("nan", "none"):
        continue

    # 1) Försök SPARQL exakt label (svenska etiketter)
    if name in cache_exact:
        candidates = cache_exact[name]
    else:
        try:
            candidates = sparql_exact_label_search(name, lang="sv")
        except Exception as e:
            # Vid fel, logga och fortsätt med API-sökning
            candidates = []
        cache_exact[name] = candidates

    chosen = None
    method = ""
    score = 0.0

    # Om vi fått kandidater från exakt label, använd första och sätt score = 1.0 (perfekt match)
    if candidates:
        chosen = candidates[0]
        method = "sparql_exact_label"
        score = 1.0
    else:
        # 2) Använd Wikidata API-sökning och beräkna likhet
        if name in cache_api:
            api_candidates = cache_api[name]
        else:
            try:
                api_candidates = wikidata_api_search(name, lang="sv", limit=10)
            except Exception as e:
                api_candidates = []
            cache_api[name] = api_candidates

        # Rangordna via text-similarity mellan namn och candidate label (sv)
        best = None
        best_score = 0.0
        for c in api_candidates:
            cand_label = c.get("label") or ""
            s = similarity(name, cand_label)
            if s > best_score:
                best = c
                best_score = s

        # Sätt tröskel för automatiskt godkännande; justera efter behov
        if best and best_score >= 0.80:
            chosen = best
            method = "api_label_highsim"
            score = best_score
        elif best and best_score >= 0.60:
            # Låg till medelhög risk — markera som "candidate_low_confidence"
            chosen = best
            method = "api_label_lowsim"
            score = best_score
        else:
            chosen = None

    # Spara resultat i df
    if chosen:
        df.at[i, "wikidata_qid"] = chosen.get("qid", "")
        df.at[i, "wikidata_label"] = chosen.get("label", "")
        df.at[i, "match_method"] = method
        df.at[i, "match_score"] = round(float(score), 3)
    else:
        df.at[i, "match_method"] = "no_match"
        df.at[i, "match_score"] = 0.0

# %% Spara output
OUT_CSV = "Myndighetsreg_alla_with_wikidata.csv"
df.to_csv(OUT_CSV, index=False)
print("Sparat:", OUT_CSV)

# %% Tips & nästa steg (körs ej):
# - Granska rader med match_method == 'api_label_lowsim' manuellt.
# - Skriv en extra kolumn med kandidatlista (flera möjliga Q-id) om du vill underlätta manuell granskning.
# - För bättre träffsäkerhet: kombinera med andra fält (organisationsnr) och kontrollera externa identifierare i Wikidata.
# - Spara mellansteg (cache_exact/cache_api) som JSON om du vill återanvända sökresultat.
# - För större mängder, använd backoff och respektfull rate-limiting (Query Service har begränsningar).
#
# Exempel för att hämta item-url för en Q-id: f"https://www.wikidata.org/wiki/{qid}"
#
# Anpassningar:
# - Om du vill matcha på engelska eller flera språk, anropa sparql_exact_label_search(name, lang="en")
# - Byt tröskelvärden (0.80/0.60) efter empiriska tester.

Matching rows: 100%|██████████████████████████| 455/455 [11:17<00:00,  1.49s/it]

Sparat: Myndighetsreg_alla_with_wikidata.csv


In [8]:
df_matched = df.copy()       # resultatet efter matchning 
df_matched["match_method"].unique()

array(['sparql_exact_label', 'no_match', 'api_label_highsim',
       'api_label_lowsim', ''], dtype=object)

### != sparql_exact_label

In [13]:
df_not_sparql_exact_label = df_matched[
    df_matched["match_method"].isna() |
    (df_matched["match_method"] != "sparql_exact_label")
]



In [25]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [26]:
df_not_sparql_exact_label

,Organisationsnr,Namn,PostAdress,PostNr,PostOrt,BesöksAdress,BesöksPostNr,BesöksPostOrt,Tfn,Fax,Epost,Webbadress,SFS,Land,Ambassadör/Generalkonsul,PostAdress1,PostAdress2,PostAdress3,PostAdress4,PostAdress5,PostAdress6,PostAdress7,BesöksAdress1,BesöksAdress2,BesöksAdress3,BesöksAdress4,BesöksAdress5,BesöksAdress6,BesöksAdress7,Sort,wikidata_qid,wikidata_label,match_method,match_score
1,202100-3625,ALLMÄNNA REKLAMATIONSNÄMNDEN,BOX 174,101 23,STOCKHOLM,Sveavägen 31,111 34,STOCKHOLM,850886000.0,NaN,arn@arn.se,www.arn.se,2015:739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,no_match,0.0
3,202100-2114,ARBETSFÖRMEDLINGEN,NaN,107 67,STOCKHOLM,ELEKTROGATAN 4,171 54,SOLNA,771600053.0,NaN,registrator@arbetsformedlingen.se,www.arbetsformedlingen.se,2022:811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q98408037,Arbetsförmedlingen,api_label_highsim,1.0
4,202100-3476,ARBETSGIVARVERKET,Mäster Samuelsgatan 60 9 tr,111 21,STOCKHOLM,Mäster Samuelsgatan 60,111 21,STOCKHOLM,NaN,NaN,registrator@arbetsgivarverket.se,www.arbetsgivarverket.se,2007:829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q79005766,Arbetsgivarverket informerar,api_label_lowsim,0.756
5,202100-2148,ARBETSMILJÖVERKET,BOX 9082,171 09,SOLNA,SVETSARVÄGEN 12 2TR,171 41,SOLNA,107309000.0,NaN,arbetsmiljoverket@av.se,www.av.se,2007:913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q123672182,Arbetsmiljöverket,api_label_highsim,1.0
7,202100-3690,BARNOMBUDSMANNEN,BOX 22106,104 22,STOCKHOLM,LINDHAGENSGATAN 126,112 51,STOCKHOLM,86922950.0,NaN,info@barnombudsmannen.se,www.barnombudsmannen.se,2007:1021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q97979799,Barnombudsmannen,api_label_highsim,1.0
8,202100-4011,BLEKINGE TEKNISKA HÖGSKOLA,NaN,371 79,KARLSKRONA,VALHALLAVÄGEN 1,371 41,KARLSKRONA,455385000.0,NaN,registrator@bth.se,www.bth.se,1993:100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q97785005,Blekinge Tekniska Högskola,api_label_highsim,1.0
9,202100-3278,BOKFÖRINGSNÄMNDEN,BOX 7849,103 99,STOCKHOLM,Sveavägen 44,111 34,STOCKHOLM,840898990.0,NaN,bfn@bfn.se,www.bfn.se,2017:153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,no_match,0.0
10,202100-5489,BOLAGSVERKET,NaN,851 81,SUNDSVALL,STUVARVÄGEN 21,852 29,SUNDSVALL,771670670.0,NaN,bolagsverket@bolagsverket.se,www.bolagsverket.se,2007:1110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q99295460,Bolagsverket,api_label_highsim,1.0
11,202100-3989,BOVERKET,BOX 534,371 23,KARLSKRONA,Skeppsbrokajen 16,371 33,KARLSKRONA,NaN,NaN,registraturen@boverket.se,www.boverket.se,2022:208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q111683594,Boverket,api_label_highsim,1.0
12,202100-0068,BROTTSFÖREBYGGANDE RÅDET,BOX 1386,111 93,STOCKHOLM,FLEMINGGATAN 14,112 26,STOCKHOLM,852758400.0,NaN,registrator@bra.se,www.bra.se,2016:1201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q97999422,Brottsförebyggande rådet,api_label_highsim,1.0


### api_label_highsim

In [24]:
df_api_label_highsim = df_matched[
    df_matched["match_method"].isna() |
    (df_matched["match_method"] == "api_label_highsim")
]
df_api_label_highsim [
    ["Organisationsnr", "Namn", "wikidata_qid", "wikidata_label", "match_method", "match_score"]
]

,Organisationsnr,Namn,wikidata_qid,wikidata_label,match_method,match_score
3,202100-2114,ARBETSFÖRMEDLINGEN,Q98408037,Arbetsförmedlingen,api_label_highsim,1.0
5,202100-2148,ARBETSMILJÖVERKET,Q123672182,Arbetsmiljöverket,api_label_highsim,1.0
7,202100-3690,BARNOMBUDSMANNEN,Q97979799,Barnombudsmannen,api_label_highsim,1.0
8,202100-4011,BLEKINGE TEKNISKA HÖGSKOLA,Q97785005,Blekinge Tekniska Högskola,api_label_highsim,1.0
10,202100-5489,BOLAGSVERKET,Q99295460,Bolagsverket,api_label_highsim,1.0
11,202100-3989,BOVERKET,Q111683594,Boverket,api_label_highsim,1.0
12,202100-0068,BROTTSFÖREBYGGANDE RÅDET,Q97999422,Brottsförebyggande rådet,api_label_highsim,1.0
14,202100-6909,CENTRALA DJURFÖRSÖKSETISKA NÄMNDEN,Q10444871,Centrala djurförsöksetiska nämnden,api_label_highsim,1.0
15,202100-1819,CENTRALA STUDIESTÖDSNÄMNDEN,Q99384445,Centrala studiestödsnämndens service,api_label_highsim,0.857
20,202100-4979,EKOBROTTSMYNDIGHETEN,Q97910560,Ekobrottsmyndigheten,api_label_highsim,1.0


In [30]:
# no_match 
df_no_match  = df[
    (df["match_method"] == "no_match")
]
df_no_match [
    ["Organisationsnr", "Namn", "wikidata_qid", "wikidata_label", "match_method", "match_score"]
]

,Organisationsnr,Namn,wikidata_qid,wikidata_label,match_method,match_score
1,202100-3625,ALLMÄNNA REKLAMATIONSNÄMNDEN,,,no_match,0.0
9,202100-3278,BOKFÖRINGSNÄMNDEN,,,no_match,0.0
22,202100-4466,ELSÄKERHETSVERKET,,,no_match,0.0
26,202100-4870,FASTIGHETSMÄKLARINSPEKTIONEN,,,no_match,0.0
29,202100-5687,FINANSPOLITISKA RÅDET,,,no_match,0.0
32,202100-7030,FONDTORGSNÄMNDEN,,,no_match,0.0
34,202100-5232,"FORSKNINGSRÅDET F MILJÖ, AREELLA NÄRINGAR OCH ...",,,no_match,0.0
35,202100-5240,"FORSKNINGSRÅDET FÖR HÄLSA, ARBETSLIV OCH VÄLFÄRD",,,no_match,0.0
45,202100-4334,Gymnastik- och idrottshögskolan (GIH),,,no_match,0.0
47,202100-4250,HARPSUNDSNÄMNDEN,,,no_match,0.0


In [31]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-11-25 16:57:16
Total time elapsed: 39 minutes 33.51 seconds
